# Engine usage

## Engine

An **engine** is the core component that establishes a connection between your Python application and a database. It acts as the interface for issuing SQL commands and managing the communication with the database.

In [1003]:
# Creating an engine

from sqlalchemy import Engine, create_engine

# Make a engine that will connect to SQLite in-memory database.
# echo=True turn SQLAlchemy logging on.
engine: Engine = create_engine("sqlite://", echo=True)

print(engine)

Engine(sqlite://)


- The Engine is a **factory** that makes new connection when used.

- The engine also stores a **connection pool**, where connections we use will be reused for subsequent operations.

- We didn't actually connect yet! We need to use the `connect()` method for this. 

## Connection

In [1004]:
from sqlalchemy import Connection


connection: Connection = engine.connect()

print(connection)

- The connection is a **proxy** for a DBAPI connection (⚠️ same word used for 2 different things)
- In this case, the DBAPI is Python's sqlite3 module.

In [1005]:
print(connection.connection.driver_connection)

## Execute

- The SQLAlchemy connection feature an `execute()` method to run queries, using the underlying DBAPI connection and cursor behind the scenes
- To invoke a textual query, we use the `sqlalchemy.text()` construct, passed to `execute()`.

In [1006]:
from sqlalchemy import text

stmt_1 = text("CREATE TABLE users (id INTEGER, name TEXT)")
stmt_2 = text("INSERT INTO users (id, name) VALUES (1, 'Alice'), (2, 'Bob')")
connection.execute(stmt_1)
result = connection.execute(stmt_2)
print(result)

2024-12-07 18:23:53,671 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 18:23:53,672 INFO sqlalchemy.engine.Engine CREATE TABLE users (id INTEGER, name TEXT)
2024-12-07 18:23:53,673 INFO sqlalchemy.engine.Engine [generated in 0.00158s] ()
2024-12-07 18:23:53,673 INFO sqlalchemy.engine.Engine INSERT INTO users (id, name) VALUES (1, 'Alice'), (2, 'Bob')
2024-12-07 18:23:53,674 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()


- The returned result is a `CursorResult`, a `Result` that is representing state from a DBAPI cursor.
- `Result` and `CursorResult` have methods, for example to fetch rows

### Example: `first()` method

Return the first row, or None is no row, and close the result set.

In [1007]:
from typing import Any
from sqlalchemy import Row

result = connection.execute(text("SELECT * from users"))
row: Row[Any] | None = result.first()

2024-12-07 18:23:53,680 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,681 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()


### The Row object

- Looks and acts mostly like a named tuple:

In [1008]:
if row:
    print(row[1])  # access the "name" column with index
    print(row.name)  # same thing using named tuple

Alice
Alice


- We can also have a dictionnary interface, available with an accessors called `_mapping`:

In [1009]:
if row:
    print(row._mapping)  # type: ignore (access to protected attribute)

{'id': 1, 'name': 'Alice'}


### What Does "Closing the Result Set" Mean?

- When a result set is closed, the underlying database cursor and connection resources associated with the query are released. This prevents resource leaks, especially in long-running applications.
Inaccessibility of Remaining Results:

- Once the result set is closed, you can **no longer fetch additional rows** or iterate over the results. The result set is finalized and discarded.

- SQLAlchemy handles this automatically with first() and similar convenience methods, so you don't need to manually manage the lifecycle of the result set.

In [1010]:
# Attempting to fetch more data will fail because the result set is closed
try:
    for row in result:
        print(row)
except Exception as e:
    print("Error:", e)

Error: This result object is closed.


### Different patterns to iterate over rows

#### Iterating on rows

In [1011]:
result = connection.execute(text("SELECT * from users"))  # reset the result

for row in result:
    print(row)

2024-12-07 18:23:53,704 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,705 INFO sqlalchemy.engine.Engine [cached since 0.02489s ago] ()
(1, 'Alice')
(2, 'Bob')


#### Iterating with tuple assignment

In [1012]:
result = connection.execute(text("SELECT * from users"))  # reset the result

for id, name in result:
    print(f"The user {id} is named {name}.")

2024-12-07 18:23:53,711 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,712 INFO sqlalchemy.engine.Engine [cached since 0.03157s ago] ()
The user 1 is named Alice.
The user 2 is named Bob.


#### Iterating through the first column of each row

In [1013]:
result = connection.execute(text("SELECT * from users"))  # reset the result

# the `scalars()`` method extracts the **first** column of each row from the query result
# when the query returns multiple columns. Cf. more explications below
for id in result.scalars():
    print(id)

2024-12-07 18:23:53,717 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,718 INFO sqlalchemy.engine.Engine [cached since 0.03796s ago] ()
1
2


#### Getting all rows in a list

In [1014]:
result = connection.execute(text("SELECT * from users"))  # reset the result

result.all()

2024-12-07 18:23:53,725 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,727 INFO sqlalchemy.engine.Engine [cached since 0.04654s ago] ()


[(1, 'Alice'), (2, 'Bob')]

#### Getting all first columns in a list

In [1015]:
result = connection.execute(text("SELECT * from users"))  # reset the result

result.scalars().all()

2024-12-07 18:23:53,735 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,737 INFO sqlalchemy.engine.Engine [cached since 0.05715s ago] ()


[1, 2]

### More about `scalars()` method

- The `scalars()` method is a streamlined way to extract individual column values from a result set.
- It is commonly used when your query returns rows but you're only interested in the values from a single column.
- It returns an iterable, which can be converted to lists, used in loops, or consumed lazily.

In [1016]:
result = connection.execute(text("SELECT * from users"))

for item in result:
    print(result)  # KO: does not display my items

2024-12-07 18:23:53,787 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,789 INFO sqlalchemy.engine.Engine [cached since 0.1088s ago] ()


In [1017]:
result = connection.execute(text("SELECT * from users"))

for item in result.scalars():
    print(item)  # OK: prints items

2024-12-07 18:23:53,796 INFO sqlalchemy.engine.Engine SELECT * from users
2024-12-07 18:23:53,797 INFO sqlalchemy.engine.Engine [cached since 0.1166s ago] ()
1
2


### Closing connections

- `close()` method **releases** the DBADI connection back to the connection pool.
    - "releases" means the pool may hold onto the connection, ot close it if it's part of overflow (over the limit)
- To avoid unintended behavior or side effects, SQLAlchemy automatically issues a ROLLBACK to discard any uncommitted changes. And After close() is called, the connection is returned to the pool. SQLAlchemy ensures the connection is clean and ready for the next use by rolling back any active transaction.

In [1018]:
connection.close()

2024-12-07 18:23:53,802 INFO sqlalchemy.engine.Engine ROLLBACK


- Modern us should favor **context managers** to manage the connect/release process instead of calling `close()` method:

In [1019]:
with engine.connect() as connection:
    result = connection.execute(text("select 'toto'"))
    print(result.all())

2024-12-07 18:23:53,810 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 18:23:53,811 INFO sqlalchemy.engine.Engine select 'toto'
2024-12-07 18:23:53,811 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ()
[('toto',)]
2024-12-07 18:23:53,813 INFO sqlalchemy.engine.Engine ROLLBACK


- If you have lots of thing to do with a connection, create a function and pass the connection to it.
- When you deal with SQLAlchemy objects, you want to make sure that when opening something, you release it correctly. Lots of problems can come from this.
- As we can see in the logs, the context managers handle the whole process with a BEGIN and a ROLLBACK in the end

## Transactions, committing

- SQLAlchemy always assumes an explicit or implicit "begin" of a transaction, at the start of some SQL operations. This behavior ensures **consistency**, as all operations are part of a transaction and subject to rollback or commit.

- Transaction Lifecycle: a transaction remains open until:
  - You explicitly commit it (`commit()`) or roll it back (`rollback()`).
  - SQLAlchemy closes the connection or session, at which point it issues a ROLLBACK to discard uncommitted changes.

- SQLAlchemy never COMMIT implicitely. It expects the user to be explicit about this.
- Note: I did not `commit()` in previous example but uncommitted transactions are visible within the same connection, in sqlite.

### Two styles to start a transaction

#### Style 1: "Commit as you go"

You write operations and commit when you are ready.

In [1022]:
with engine.connect() as connection:
    #  .. create table
    # .. insert row
    connection.commit()

#### Style 2: "begin once"

Here the transaction start as an explicit block that commits when complete. Here you want to run a single transaction.

In [1021]:
with engine.begin() as connection:
    # .. create table
    # .. insert row
    ...

# end of block: commits transaction, releases connection back to the connection pool.
# rolls back if there is an exception

2024-12-07 18:23:53,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 18:23:53,825 INFO sqlalchemy.engine.Engine COMMIT
